<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-13 13:01:14
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.33 C
-------------------
Today PnL: -1.40 L (-1.04%)
Current PnL: -17.04 L (-11.87%)
CY Booked + Current PnL: -10.67 L (-7.44%)
-------------------
Total profit:  2.80 L
Total loss:  -19.84 L
-------------------
Total Booked + Current PnL: 16.31 L (13.99%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.79%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.94 L (54.87%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-5.02,H-LC,87.91,96511.0,11324.0,4739.0,-0.71,13.29,4.91,18.85,35.0,2.39,0.75,28.50,XY25,NTT,INSURANCE
60,RELIANCE,1291.83,1526.0,4.98,X-LC,38.46,155881.0,15072.0,10460.0,-0.80,10.70,6.71,18.13,17.0,1.44,1.20,23.08,XY25,NTT,REFINERIES
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,82.42,132354.0,-10125.0,10747.0,1.26,-7.11,8.12,0.44,242.0,-0.94,1.02,56.21,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,46.15,99428.0,21066.0,12965.0,-2.67,26.88,13.04,43.43,77.0,1.62,0.77,33.63,XY24,NTT,MISC
76,TATAELXSI,7332.28,7332.0,-14.80,X-MC,69.23,89530.0,-13122.0,13116.0,-1.21,-12.78,14.65,-0.00,57.0,-1.00,0.69,36.10,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,561.33,680.00,-16.65,H-MC,52.20,228724.0,1385.0,46683.0,-0.67,0.61,20.41,21.14,117.0,0.03,1.77,28.06,XY24,NTT,PAINTS
32,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,58.24,150504.0,-409.0,60864.0,-1.44,-0.27,40.44,40.06,100.0,-0.01,1.16,23.10,AR,ATH,AUTO
88,VOLTAS,1278.28,1929.20,-29.62,H-MC,42.86,191835.0,93.0,97548.0,-0.85,0.05,50.85,50.92,99.0,0.00,1.48,6.26,XY25,ATH,AC
56,PGHH,13388.00,18106.42,-20.14,X-MC,13.19,201540.0,720.0,70055.0,0.26,0.36,34.76,35.24,53.0,0.01,1.56,5.05,X40,ATH,FMCG
19,CIPLA,1495.00,1795.00,-17.66,H-LC,23.08,100621.0,456.0,19641.0,-0.07,0.46,19.52,20.07,29.0,0.02,0.78,8.46,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7227.78,M-SC,100.00,80387.0,-13423.0,90041.0,4.01,-14.31,112.01,81.67,238.0,-0.15,0.62,55.67,XR,NTT,CERAMICS
82,UJJIVANSFB,52.77,53.00,52.86,M-SC,82.42,132354.0,-10125.0,10747.0,1.26,-7.11,8.12,0.44,242.0,-0.94,1.02,56.21,OX40N,NTT,BANKS
37,ICICIGI,1839.64,2260.25,-13.82,H-MC,47.25,142028.0,5895.0,25224.0,0.91,4.33,17.76,22.86,34.0,0.23,1.10,17.49,X40,ATH,INSURANCE
0,5PAISA,564.75,565.00,150.00,H-SC,62.64,135131.0,-51801.0,51890.0,0.62,-27.71,38.40,0.04,149.0,-1.00,1.04,29.27,OX40N,NTT,FINANCE
51,LTIM,5564.16,7197.33,1.86,H-LC,81.32,195894.0,-4416.0,63215.0,0.49,-2.20,32.27,29.35,36.0,-0.07,1.51,35.33,X200,ATH,IT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,SURYODAY,189.78,240.00,57.89,H-SC,80.22,112172.0,-37754.0,77432.0,-3.78,-25.18,69.03,26.46,143.0,-0.49,0.87,43.70,XR,NTT,BANKS
58,RAJESHEXPO,517.65,518.00,1777.53,M-SC,29.67,51214.0,-85963.0,86055.0,-3.67,-62.67,168.03,0.07,232.0,-1.00,0.40,27.37,OX40N,NTT,JEWELLERY
59,RELAXO,902.64,1176.00,-40.46,H-SC,24.18,53862.0,-61676.0,96666.0,-3.05,-53.38,179.47,30.28,135.0,-0.64,0.42,5.23,X40N,NTT,FOOTWEAR
47,KANSAINER,299.63,340.00,-68.58,H-SC,7.69,218430.0,-51237.0,87569.0,-2.86,-19.00,40.09,13.47,144.0,-0.59,1.69,9.33,XY24,NTT,PAINTS
75,TANLA,917.30,1963.11,-29.78,H-SC,93.41,172325.0,-67090.0,340049.0,-2.79,-28.02,197.33,114.01,128.0,-0.20,1.33,58.05,AR,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,82.42,132354.0,-10125.0,10747.0,1.26,-7.11,8.12,0.44,242.0,-0.94,1.02,56.21,OX40N,NTT,BANKS
76,TATAELXSI,7332.28,7332.0,-14.80,X-MC,69.23,89530.0,-13122.0,13116.0,-1.21,-12.78,14.65,-0.00,57.0,-1.00,0.69,36.10,OX40N,NTT,IT
74,SYMPHONY,1306.42,1306.0,-30.13,M-SC,2.20,148816.0,-22325.0,22263.0,-2.37,-13.04,14.96,-0.03,190.0,-1.00,1.15,6.96,OX40N,NTT,DURABLES
87,VIPIND,488.80,489.0,-930.51,H-SC,91.21,79092.0,-16224.0,16261.0,-2.68,-17.02,20.56,0.04,153.0,-1.00,0.61,56.00,OX40N,NTT,MISC
18,CERA,8421.60,8422.0,-12.41,X-SC,63.74,90500.0,-18981.0,18987.0,-0.63,-17.34,20.98,0.00,65.0,-1.00,0.70,36.97,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-10.62,X-LC,48.35,287009.0,25408.0,120228.0,-0.32,9.71,41.89,55.67,1.0,0.21,2.22,16.41,X40,BTT,IT
78,TCS,3794.03,4998.00,-17.76,X-LC,19.78,268226.0,-27708.0,121614.0,0.13,-9.36,45.34,31.73,2.0,-0.23,2.07,7.36,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-17.43,X-LC,16.48,259358.0,-10989.0,45621.0,-0.73,-4.06,17.59,12.81,5.0,-0.24,2.00,7.34,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,10.87,X-LC,34.07,254380.0,25147.0,42151.0,-0.71,10.97,16.57,29.35,7.0,0.60,1.97,20.87,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-12.85,X-LC,26.37,276939.0,11513.0,45390.0,-0.78,4.34,16.39,21.44,10.0,0.25,2.14,10.80,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,46.15,99428.0,21066.0,12965.0,-2.67,26.88,13.04,43.43,77.0,1.62,0.77,33.63,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,46.15,99428.0,21066.0,12965.0,-2.67,26.88,13.04,43.43,77.0,1.62,0.77,33.63,XY24,NTT,MISC
17,CAMS,3643.00,5250.99,-1.11,H-SC,71.43,111096.0,9092.0,35928.0,-2.58,8.91,32.34,44.14,125.0,0.25,0.86,26.60,X40N,ATH,MISC
83,UNIONBANK,123.87,163.00,-4.91,M-LC,84.62,166798.0,25958.0,18531.0,-2.52,18.43,11.11,31.59,86.0,1.40,1.29,44.89,XY24,NTT,BANKS
90,WIPRO,243.46,420.00,-5.63,M-LC,49.45,161696.0,10751.0,98699.0,0.46,7.12,61.04,72.51,68.0,0.11,1.25,11.41,XR,NTT,IT
38,ICICIPRULI,600.83,790.00,-12.62,H-MC,50.55,141250.0,5462.0,37290.0,-1.05,4.02,26.40,31.48,119.0,0.15,1.09,16.63,X40,ATH,INSURANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-32.39,M-SC,59.34,197259.0,25638.0,136424.0,0.13,14.94,69.16,94.43,194.0,0.19,1.52,45.59,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-5.63,M-LC,49.45,161696.0,10751.0,98699.0,0.46,7.12,61.04,72.51,68.0,0.11,1.25,11.41,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-29.44,M-MC,64.84,185898.0,6090.0,106315.0,-1.55,3.39,57.19,62.52,170.0,0.06,1.44,33.78,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.97,H-SC,56.04,130380.0,7044.0,127785.0,-1.36,5.71,98.01,109.32,122.0,0.06,1.01,28.81,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,58.24,150504.0,-409.0,60864.0,-1.44,-0.27,40.44,40.06,100.0,-0.01,1.16,23.10,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.70,68352.0,-45197.0,85249.0,-0.50,-39.80,124.72,35.27,260.0,-0.53,0.53,67.29,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-23.67,H-SC,70.33,86775.0,-11205.0,33079.0,-1.63,-11.44,38.12,22.32,141.0,-0.34,0.67,25.85,XR,ATH,FINANCE
50,LICI,827.06,983.00,-5.02,H-LC,87.91,96511.0,11324.0,4739.0,-0.71,13.29,4.91,18.85,35.0,2.39,0.75,28.50,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,46.15,99428.0,21066.0,12965.0,-2.67,26.88,13.04,43.43,77.0,1.62,0.77,33.63,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,37.93,H-LC,95.60,103160.0,200.0,45793.0,-0.28,0.19,44.39,44.67,25.0,0.00,0.80,33.13,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7227.78,M-SC,100.0,80387.0,-13423.0,90041.0,4.01,-14.31,112.01,81.67,238.0,-0.15,0.62,55.67,XR,NTT,CERAMICS
85,VALIANTORG,512.64,838.00,-392.58,H-SC,98.9,124306.0,-8980.0,93578.0,-2.53,-6.74,75.28,63.47,142.0,-0.10,0.96,108.37,XR,NTT,CHEMICALS
46,JPPOWER,18.73,26.20,-34.40,L-SC,97.8,133133.0,-8709.0,65275.0,-0.85,-6.14,49.03,39.88,258.0,-0.13,1.03,34.92,XY24,NTT,POWER
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.7,68352.0,-45197.0,85249.0,-0.50,-39.80,124.72,35.27,260.0,-0.53,0.53,67.29,XR,NTT,HOTELS
35,HINDZINC,514.80,744.74,37.93,H-LC,95.6,103160.0,200.0,45793.0,-0.28,0.19,44.39,44.67,25.0,0.00,0.80,33.13,X5K,ATH,METALS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,43.24
2,50,71.99


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.87
XY25     14.29
X40      12.63
XR       11.05
OX40N     8.52
X40N      8.50
AR        8.10
X5K       2.35
X200      1.51
SR        1.20
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.67
X-LC    15.62
M-SC    14.40
H-LC    11.14
H-MC     9.01
M-LC     8.50
X-MC     7.71
M-MC     5.80
X-SC     2.82
L-SC     2.49
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.69,-3.11,37.61
IT,11.46,-11.88,68.69
BANKS,8.53,-10.32,54.78
MISC,6.59,-26.97,76.62
FINANCE,6.38,-21.77,68.63
PAINTS,5.66,-19.34,42.86
ELECTRICAL,5.15,-5.60,51.40
HEALTHCARE,4.77,-4.19,34.62
AUTO,4.67,-11.79,60.95


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2805512.0,25
XR,1131992.0,13
AR,1023376.0,9
X40,686589.0,9
XY25,538784.0,10
X40N,430677.0,8
OX40N,327087.0,11
SR,188851.0,2
X5K,97684.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1953003.0,19
M-SC,1463265.0,18
X-LC,643666.0,9
M-MC,593941.0,5
H-MC,560707.0,7
H-LC,513437.0,10
X-MC,419836.0,6
M-LC,344647.0,6
L-SC,275693.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,920430.0,6
M-SC,XY24,587299.0,5
H-SC,AR,522630.0,3
M-MC,XY24,399076.0,3
M-SC,XR,344466.0,4
X-LC,X40,325743.0,3
H-MC,XY24,227900.0,2
M-LC,XY24,226969.0,4
H-SC,XR,204089.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
